# Personal Search

## Multi Query Answering System



## Sistema de resposta Multi Query



## Import libraries

In [1]:
from IPython.display import display, Markdown
from langchain import hub
from search.context import utils

from operator import itemgetter
#from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

## Configuration

In [2]:
# Model from Ollama
model = "llama3:8b"

## Retriever Instance

In [3]:
retriever, db = utils.retriver_context()

## Prompt

In [4]:
# Multi Query: Different Perspectives
prompt_perspectives = hub.pull(owner_repo_commit="personal-search/multi-query-search")

## Question

In [5]:
question = "O que falta aos pregadores de hoje?"

## Search

In [6]:
generate_queries = utils.generate_multi_query(prompt=prompt_perspectives)

queries_pespectives = generate_queries.invoke({"question":question})

In [1]:
queries_pespectives

NameError: name 'queries_pespectives' is not defined

In [8]:
# Retrieve
retrieval_chain = generate_queries | retriever.map() | utils.get_unique_union
#docs = retrieval_chain.invoke({"question":question})


In [24]:
# RAG
template = """
Contexto:
{context}

Sintetize o contexto respondendo a seguinte pergunta e cite a fonte(source): {question}
"""

prompt = ChatPromptTemplate.from_template(template)

#llm = ChatOpenAI(temperature=0)
llm = Ollama(model="llama3:8b", temperature=0)

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm.bind(stop=['<|eot_id|>'])
    | StrOutputParser()
)

final_rag_answer = final_rag_chain.invoke({"question":question})

## Anwser

In [25]:
display(Markdown(f"**Busca:** {question}\n\n"))
display(Markdown(final_rag_answer))

**Busca:** O que falta aos pregadores de hoje?



De acordo com Leonard Ravenhill, em seu livro "Por que tarda o pleno Avivamento", os pregadores de hoje carecem de coração quebrantado, pois apenas quem tem um coração quebrantado é capaz de levar outros ao quebrantamento. Além disso, eles têm equipamentos, mas não tem poder; têm ação, mas não unção; barulho, mas não avivamento. (Fonte: "Por que tarda o pleno Avivamento" - Leonard Ravenhill, página 77)

## Debug

In [26]:
sources = retriever.invoke(question)
for source in sources:
    display(Markdown(f"**Fonte:** {source.metadata}"))
    display(Markdown(f"**Trecho:** {source.page_content}"))

**Fonte:** {'source': './books/import/Por que tarda o pleno Avivamento - Leonard Ravenhill.pdf', 'page': 45}

**Trecho:** pregadores, precisamos nos enver gonhar de não sentir vergonha; 
precisamos chorar por n ão termos lágrimas; precisamos nos humilhar 
por haver perdido a humildade de servo de Deus; gemer por não 
sentirmos peso pelos perdi dos; irar -nos contra n ós mesmos por não 
termos ódio do monopólio que o diabo exerce nestes dias do fim, e 
nos punir pelo fato de o mundo estar -se dando tão bem conosco, que 
nem precisa perseguir -nos. 
Pentecostes significa dor, mas o que ma is expe rimentamos  é 
prazer; significa peso; mas nós amamos a comodidade. Pentecostes 
significa prisão, e, no entanto, a maioria dos crentes faria qualquer 
coisa, menos ir para a prisão por amor a Cristo. Se revivêss emos a 
experiência do pentecostes, talvez muitos de nós fossem parar na

**Fonte:** {'source': './books/import/Por que tarda o pleno Avivamento - Leonard Ravenhill.pdf', 'page': 77}

**Trecho:** na sua fidelidade ao Pai  (prepa rou-se durante tanto tempo pa ra 
pregar por tão curto período); grande em sua submissão  ao Espírito 
(anda va ou parava de acordo com as orientações dele); grande nas 
afirm ações que fez  sobre o Filho (apon tando Jesus como “o Cordeiro 
de Deus, que tira o pecado do mundo ”, apesar de não te r-se avistado 
com ele antes).  
João era uma “Voz”. A maioria dos pregadores não passa de ecos, 
pois, se prestarmos bem atenção, saberemos dizer quais os livros 
que andaram lendo, e notaremos que citaram muito pouco do Livro. E 
hoje, só uma Voz, voz de um pr ofeta enviado do céu par a pregar aos 
pregadores , conseguiria despertar o co ração dos homens. S ó quem 
tem coração quebrantado é capaz de levar outros ao 
quebrantamento. Ir mãos, nós temos equipamentos, mas não temos 
poder; temos ação, mas não unção; barulho,  mas não avivamento. 
Somos dogmát icos, mas  não dinâmicos!  
Todas as eras têm iniciado com fogo, e todas as vidas, sejam de

**Fonte:** {'source': './books/import/Conselhos para obreiros - Charles H Spurgeon.pdf', 'page': 11}

**Trecho:** “Pensei que ele não havia se preocupado muito conosco, porque sabia que, por onde quer que ele
passe, argumenta com seus anfitriões, e não havia feito assim conosco. Cheguei a pensar que
havíamos irritado o sr. Whitefield, mas agora percebo como ele foi amável conosco, falando-nos
dessa maneira.”
Em seguida chamou suas filhas: “Subam, meninas. Vejam o que o homem de Deus escreveu na
vidraça: ‘Falta-lhes uma coisa!’. Chamem seu pai”. O coronel subiu ao aposento de hóspedes e
também leu a frase: “Falta-lhes uma coisa!”. Assim, ao redor da cama onde o homem de Deus
havia dormido, ajoelharam-se e pediram a Deus que lhes desse a coisa que faltava. E, ali mesmo,
antes  que deixassem o quarto, encontraram  o que lhes  faltava;  e toda aquela  família  pôde
regozijar-se em Jesus.
Não faz muito tempo, encontrei-me com um amigo em cuja igreja há uma irmã que preserva
aquele pedaço de vidro como herança de família. Se você não conseguir aconselhar ou admoestar

In [18]:
sources = db.similarity_search_with_score(question)
for source in sources:
    display(Markdown(f"**Fonte:** {source[0].page_content}"))
    display(Markdown(f"**Trecho:** {source[0].metadata}"))
    display(Markdown(f"**Score:** {source[1]}"))    

**Fonte:** pregadores, precisamos nos enver gonhar de não sentir vergonha; 
precisamos chorar por n ão termos lágrimas; precisamos nos humilhar 
por haver perdido a humildade de servo de Deus; gemer por não 
sentirmos peso pelos perdi dos; irar -nos contra n ós mesmos por não 
termos ódio do monopólio que o diabo exerce nestes dias do fim, e 
nos punir pelo fato de o mundo estar -se dando tão bem conosco, que 
nem precisa perseguir -nos. 
Pentecostes significa dor, mas o que ma is expe rimentamos  é 
prazer; significa peso; mas nós amamos a comodidade. Pentecostes 
significa prisão, e, no entanto, a maioria dos crentes faria qualquer 
coisa, menos ir para a prisão por amor a Cristo. Se revivêss emos a 
experiência do pentecostes, talvez muitos de nós fossem parar na

**Trecho:** {'source': './books/import/Por que tarda o pleno Avivamento - Leonard Ravenhill.pdf', 'page': 45}

**Score:** 398.0030822753906

**Fonte:** na sua fidelidade ao Pai  (prepa rou-se durante tanto tempo pa ra 
pregar por tão curto período); grande em sua submissão  ao Espírito 
(anda va ou parava de acordo com as orientações dele); grande nas 
afirm ações que fez  sobre o Filho (apon tando Jesus como “o Cordeiro 
de Deus, que tira o pecado do mundo ”, apesar de não te r-se avistado 
com ele antes).  
João era uma “Voz”. A maioria dos pregadores não passa de ecos, 
pois, se prestarmos bem atenção, saberemos dizer quais os livros 
que andaram lendo, e notaremos que citaram muito pouco do Livro. E 
hoje, só uma Voz, voz de um pr ofeta enviado do céu par a pregar aos 
pregadores , conseguiria despertar o co ração dos homens. S ó quem 
tem coração quebrantado é capaz de levar outros ao 
quebrantamento. Ir mãos, nós temos equipamentos, mas não temos 
poder; temos ação, mas não unção; barulho,  mas não avivamento. 
Somos dogmát icos, mas  não dinâmicos!  
Todas as eras têm iniciado com fogo, e todas as vidas, sejam de

**Trecho:** {'source': './books/import/Por que tarda o pleno Avivamento - Leonard Ravenhill.pdf', 'page': 77}

...

**Trecho:** {'source': './books/import/Conselhos para obreiros - Charles H Spurgeon.pdf', 'page': 11}

**Score:** 410.0894775390625

**Fonte:** ela mesma. 
Se Jesus tivesse dito a ela antes de fazê-la: "Há apenas duas vagas disponíveis para você. Você pode ser uma negra ou então uma branca pobre. Qual dessas?", ela teria se retraído e se contorcido antes de finalmente responder: "Tudo bem, faça-me negra então, mas não uma dessas maloqueiras". E el e a teria feito uma mulher negra limpa e 
asseada, ela mesma numa versão negra. 
 Bem, um dia a sra. Turpin vai ao consultório médico e vê-se cercada pelo tipo de pessoas que ela despreza. De repente uma jovem com acne atra vessa a passos largos a sala de espera, golpeia 
a sra. Turpin com um livro e tenta estrangulá-l a. Quando a ordem é finalmente restaurada, a 
sra. Turpin, como se estivesse ag uardando uma revelação, pergunta: 
— O que você tem a me dizer? 
— Volte para o inferno de onde você ve io, sua porca velha! — grita a moça. 
A sra. Turpin desaba. O seu mundo fez-se em pedaços. O Deus que ela havia moldado à

**Trecho:** {'source': './books/import/Evangelho Maltrapilho - Brennan Manning.pdf', 'page': 18}

**Score:** 416.1697692871094